<a href="https://colab.research.google.com/github/espada105/NewRecipe/blob/main/Llama3_2_11B_Vision_Instruct_NewRecipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets accelerate

In [ ]:
from transformers import VisionEncoderDecoderModel, Trainer, TrainingArguments
from datasets import load_dataset

# 모델 불러오기
model = VisionEncoderDecoderModel.from_pretrained("meta-llama/Llama-3.2-11B-Vision-Instruct")

# 데이터셋 로드
dataset = load_dataset("path_to_your_ingredient_dataset")

# 이미지와 텍스트 전처리 함수 정의
def preprocess(batch):
    # 여기에 전처리 코드를 추가하여 batch['image']와 batch['text']를 모델이 수용할 수 있도록 변환합니다
    return batch

dataset = dataset.map(preprocess)

# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"]
)

# 학습 시작
trainer.train()
